# Unit Simulation Experimental Notebook

In [1]:
from IPython.display import HTML
%matplotlib inline
# our simulator
import battlesim as bsm

### Trial 1. 50 Clone Troopers vs. 80 B1 battledroids

Using Random AI selection for targets.

In [2]:
trial1 = bsm.Battle("datasets/starwars-clonewars.csv")
# define 100 battledroids, 50 clones
trial1.create_army([("b1 battledroid", 80), ("Clone Trooper", 20), ("Clone Trooper", 15)])
# battle droids near (0, 0), clones near (10, 10)
trial1.apply_position([
    dict(name="uniform", x_loc=-5., scale=2.),
    dict(name="gaussian", loc=0.),
    dict(name="gaussian", x_loc=4., y_loc=5.),
])
#init, rolling AIs now default to 'nearest'
trial1.set_decision_ai(["aggressive","hit_and_run","hit_and_run"])
# set boundary
trial1.set_bounds((-10., 20., -10., 20.))
# set terrain
trial1.apply_terrain("contour", res=.5)

C:\Users\Gregory\Documents\PublicCode\BattleSimulator\battlesim\_battle.py:63: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  return np.zeros(n, dtype=[


bsm.Battle(init=True, n_armies=3, simulated=False)

In [3]:
# run a battle
F = trial1.simulate(max_step=1500)

C:\Users\Gregory\Documents\PublicCode\BattleSimulator\battlesim\_simulator_fast.py:109: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  frames = np.zeros(


In [4]:
F.memory_usage()

Index         77280
army           9660
allegiance     9660
alive          9660
hp            38640
armor         38640
x             38640
y             38640
dir_x         38640
dir_y         38640
dtype: int64

In [5]:
F.memory_usage().sum()/1000000

0.3381

In [6]:
trial1.sim_jupyter()

In [7]:
# trial1.sim_export("simulations/main.gif", writer="imagemagick")

## Testing Performance Profiling

In [8]:
%timeit trial1.simulate()

C:\Users\Gregory\Documents\PublicCode\BattleSimulator\battlesim\_simulator_fast.py:109: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  frames = np.zeros(


687 ms ± 52.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### With and without hit-and-run tactics..  Wow

In [7]:
trial1.set_decision_ai(["aggressive","hit_and_run","hit_and_run"])
F = trial1.simulate_k(10,max_step=500)

trial1.set_decision_ai(["aggressive"]*3)
F2 = trial1.simulate_k(10,max_step=500)

C = pd.concat([
    F.melt(var_name="team", value_name="number of units").assign(Run="w/ hit and run"),
    F2.melt(var_name="team", value_name="number of units").assign(Run="w/o hit and run"),
], axis=0)

NameError: name 'pd' is not defined

In [ ]:
import seaborn as sns
sns.barplot(data=C, x="Run", y="number of units", hue="team")
plt.show()

In [16]:
%prun trial1.simulate()

In [ ]:
%prun trial1.simulate_k(1)